In [1]:
import os
os.chdir("..")
os.getcwd()

'D:\\Programming\\Python\\DS\\Hanoi-Real-Estate-Analysis-and-Prediction'

In [2]:
from fastapi import FastAPI, HTTPException
from joblib import load

#spam_clf = load(open('./models/spam_detector_model.pkl','rb'))

#vectorizer = load(open('./vectors/vectorizer.pickle', 'rb'))

app = FastAPI()

In [3]:
@app.get("/")
def root():
    return {"message": "Welcome to Your Sentiment Classification FastAPI"}

In [4]:
@app.post("/predict_sentiment")
def predict_sentiment(text_message):

    polarity = ""

    if(not(text_message)):
        raise HTTPException(status_code=400, 
                            detail = "Please Provide a valid text message")

    prediction = spam_clf.predict(vectorizer.transform([text_message]))

    if(prediction[0] == 0):
        polarity = "Ham"

    elif(prediction[0] == 1):
        polarity = "Spam"
        
    return {
            "text_message": text_message, 
            "sentiment_polarity": polarity
           }

In [5]:
!uvicorn app:app --reload

^C
